In [41]:
import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 3D 모델 좌표 (코, 양 눈, 양 입꼬리, 턱) → PnP에 쓸 기준점
FACE_3D_IDX = [1, 33, 263, 61, 291, 199]

def estimate_head_pose(video_path):
    cap = cv2.VideoCapture(video_path)
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as face_mesh:

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            h, w = frame.shape[:2]
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = face_mesh.process(rgb)

            if result.multi_face_landmarks:
                landmarks = result.multi_face_landmarks[0].landmark

                # ----- Landmarks 그리기 -----
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=result.multi_face_landmarks[0],
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                )
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=result.multi_face_landmarks[0],
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                )

                # ----- Head pose 계산 -----
                face_2d, face_3d = [], []
                for idx in FACE_3D_IDX:
                    x, y = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
                    face_2d.append([x, y])
                    face_3d.append([x, y, landmarks[idx].z])

                face_2d = np.array(face_2d, dtype=np.float64)
                face_3d = np.array(face_3d, dtype=np.float64)

                focal_length = 1 * w
                cam_matrix = np.array([[focal_length, 0, w / 2],
                                       [0, focal_length, h / 2],
                                       [0, 0, 1]])
                dist_matrix = np.zeros((4, 1), dtype=np.float64)

                success, rvec, tvec = cv2.solvePnP(face_3d, face_2d,
                                                   cam_matrix, dist_matrix)

                rmat, _ = cv2.Rodrigues(rvec)
                angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
                pitch, yaw, roll = angles

                # ----- 시선 방향 판정 -----
                if yaw > 0.2:
                    gaze = "오른쪽"
                elif yaw < -0.2:
                    gaze = "왼쪽"
                else:
                    gaze = "정면"

                cv2.putText(frame, f"Gaze: {gaze}", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # ----- 결과 보여주기 -----
            cv2.imshow("Head Pose Estimation with Landmarks", frame)
            if cv2.waitKey(1) & 0xFF == 27:  # ESC 누르면 종료
                break

    cap.release()
    cv2.destroyAllWindows()

In [42]:
import os

video_file = "/home/sohyunkang/asd_video/head_turn/IF2001_1_1_1023041311_1.mp4"   # 여기에 분석할 mp4 경로 입력
estimate_head_pose(video_file)

: 